## Ks from Minasny NN from efective porosity

Minasny and McBratney (2002) by using a Neuro-m method for fitting neural network PTF ontained good results in the obtaining of van Genuchten parameters:

$ \theta(h) = \theta_r+\frac{\theta_s-\theta_r}{(1+\left\lvert{\alpha h}\right\rvert^n)^m} $

where $\theta(h)$ is the volumetric water content at *h* potential, $\theta_r$ and $\theta_s$ are the residual and saturated water contents, $\alpha$ is a scaling parameter, *n* is the curve shape factor and *m* is an empirital constant equal to $1-1/n$.
The neural network applied by Minasny and McBratney (2002) was used in Australian soils, and can be used in order to obtain the efective porosity or $\phi_e$, which corresponds to the total porosity minus the water content at a potential of -33 kPa, which can be used in the PTF formulated by Forrest in the above section.

## Ks obtained from Forrest et al., (1985).

In another article of Minasny and McBratney (2000), carried out in soils of Australia, different pedotransfer functions were assessed considering the limited Ks data in Australia. In the article, the model of Forrest et al. (1985) presents a good fit even though that it was developed for a wide range of textures (clay content from 5% to 69%):

$ln K_s = 10.8731 + 3.9140 ln \phi_e$

Where $\phi_e$ corresponds to efective porosity calculated through the the substraction of total porosity and water content at -33 kPa and *Ks* to the saturated hydraulic conductivity in (mm d<sup>-1</sup>). 

In [ ]:
import gdal, osr
from gdalconst import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#inputs
Clay1=gdal.Open(r'C:/Users/ifue3702/Documents/Data Au/NSW/Clay/Clay_0-5_Namoi.tif', GA_ReadOnly)
Clay1_array=Clay1.ReadAsArray()

Sand1=gdal.Open(r'C:/Users/ifue3702/Documents/Data Au/NSW/SAnd/Sand_0-5_Namoi.tif', GA_ReadOnly)
Sand1_array=Sand1.ReadAsArray()

Bd1=gdal.Open(r'C:/Users/ifue3702/Documents/Data Au/NSW/Bd/BD_0-5_Namoi.tif', GA_ReadOnly)
Bd1_array=Bd1.ReadAsArray()

inputs1=np.dstack((Sand1_array,Clay1_array,Bd1_array)) #create array of three dimensions with the three input rasters
np.shape(inputs1)


#path to weight files
w1='C:/Users/ifue3702/Documents/PTF Budiman/Wr1_au3.txt'
w2='C:/Users/ifue3702/Documents/PTF Budiman/Wr2_au3.txt'


#reading and converting the weighted files obtained from Minasny into arrays
with open(w1) as f:  #open the first file
    data=[list(map(float, line.split())) for line in f]  #transform to list values in lines

weigh1=np.array(data)  #transform into array
weigh11=weigh1.astype(np.float)

f=open(w2, 'r')   #opend the second file
a=[]              #create empty list
for line in f:    #loop for each line of file
    a.append(line.split())   #add line into empty list and split each line
    
b=np.array(a)     #transform file into array
weigh2=b.astype(np.float)    #define values as float type


#parameters in the NN application
ni=3 #no. inputs  sand,clay,bd
no=4 #no. outputs
nh=5 # no. hidden units
nbag = 50 # no. bootstrap


#NN function
def neuro(Weight1, Weight2, X): 
    X=np.array(X, ndmin=2)     #read inputs as array of 1 row and three columns
    num_rows, num_cols=X.shape 
    N = num_rows
    ni= num_cols
    M1=np.ones(shape=(1,N))
    # from input layer to hidden layer
    h=np.dot(Weight1,np.append(X.T, M1))    #Matrix multiplication (weigth and inputs)
    y1=np.tanh(h).reshape(ni+2,N)
    # from hidden layer to output layer
    h2=np.dot(Weight2,np.append(y1, M1)).reshape(ni+1,N)   #Matrix multiplication (weigth2 and inputs)
    Y=h2.T
    return(Y)

#inputs (sand, clay, Bd) 
#X=(50.9,11,1.39)

# store parameters as array
aW1=np.zeros(shape=(nbag,nh,ni+1)) 
aW2=np.zeros(shape=(nbag,no,nh+1)) 
for ibag in range(0,nbag): # loop though  n bootstrap, fill the created matrix
    aW1[ibag,:,:]=np.array(weigh11[ibag,:].tolist()).reshape(ni+1,nh).T 
    aW2[ibag,:,:]=np.array(weigh2[ibag,:].tolist()).reshape(nh+1,no).T

'''pred=np.zeros(shape=(nbag,no))
for ibag in range(0,nbag):# loop though  n bootstrap
    Weight1=aW1[ibag,:,:]
    Weight2=aW2[ibag,:,:]
    bp = neuro(Weight1,Weight2,X) # evaluate the nnet
    bp[:,0] = bp[:,0]**2  # back transform theta_r
    bp[:,2] = np.exp(bp[:,2])  # back transform alfa
    bp[:,3] = np.exp(bp[:,3])+1  # back transform n
    pred[ibag,:]=bp     # predicted parameters

print(pred) # output is a distribution of the van Genuchten parameters (theta_r, theta_s, alfa, n)'''

inputs=inputs1
loco=np.zeros(shape=(np.shape(inputs)[0],np.shape(inputs)[1],4,nbag))#creates an array that will be filled with outputs

for [i,j,k], value in np.ndenumerate(inputs): #it will fill my array with 50 rasters for each van genuchten predicted parameter
    X=inputs[i,j,:]
    if c in X < 0:
        continue
    for ibag in range(0,nbag):# loop though  n bootstrap
        Weight1=aW1[ibag,:,:]
        Weight2=aW2[ibag,:,:]
        bp = neuro(Weight1,Weight2,X) # evaluate the nnet
        loco[i,j,0,ibag]= bp[:,0]**2  # back transform theta_r
        loco[i,j,1,ibag]= bp[:,1]
        loco[i,j,2,ibag]= np.exp(bp[:,2])  # back transform alfa
        loco[i,j,3,ibag] = np.exp(bp[:,3])+1  # back transform n
        if ibag==nbag:
            print('iteration done')
            
V_gen_mean=np.zeros(shape=(np.shape(inputs)[0], np.shape(inputs)[1], 4))#create empty array to be filled  
for [i,j,k,z], value in np.ndenumerate(loco):#return array (i,j, n° outputs) with the mean of the bootstrap calculated outputs
    V_gen_mean[i,j,k]=np.mean(loco[i,j,k,:])
    
V_gen_dev=np.zeros(shape=(np.shape(inputs)[0], np.shape(inputs)[1], 4)) #create empty array to be filled 
for [i,j,k,z], value in np.ndenumerate(loco): #return array (i,j, n° outputs) with the std of the bootstrap calculate
   
    V_gen_dev[i,j,k]=np.std(loco[i,j,k,:])


#obtaining porosity raster from Bd 
total_por1 = np.zeros(shape=(2648,5170)) #create empty array to be filled 
#function to obtain porosity
def porosity(Bulk_density, total_porosity):
    for [i,j], value in np.ndenumerate(Bulk_density):
        if Bulk_density[i,j] <= 0:
            total_porosity[i,j] = 0
        else:
            total_porosity[i,j]=1-(Bulk_density[i,j]/2.65)
            
porosity(Bd1_array, total_por1)#execute function


efective_porosity = np.zeros(shape=(np.shape(inputs)[0], np.shape(inputs)[1]))
for [i,j,k], value in np.ndenumerate(V_gen_mean): #application van Genuchten equation to each pixel to obtain effective porosity from porosity and water content at -33
    efective_porosity[i,j]=total_por1[i,j]-(V_gen_mean[i,j,0]+((V_gen_mean[i,j,1]-V_gen_mean[i,j,0])/(1+(np.absolute(V_gen_mean[i,j,2]*-33))**V_gen_mean[i,j,3])**(1-1/V_gen_mean[i,j,3])))
    
Hy_K = np.zeros(shape=(np.shape(inputs)[0], np.shape(inputs)[1]))
for [i,j], value in np.ndenumerate(efective_porosity): #application van Genuchten equation to each pixel
    if efective_porosity[i,j] < 0:
        Hy_K[i,j]=0
    else:
        Hy_K[i,j]=(np.exp(10.8731+3.9140*np.log(efective_porosity[i,j])))*24/1000

In [ ]:
'''This script transform the array to a raster image and georreferences it =)'''
#function to export array to tif
def new_array_to_raster(array, dst_filename):
    """Array > Raster
    Save a raster from a C order array.

    :param array: ndarray
    """
#file to create

    # You need to get those values like you did.
    x_pixels = 5170  # number of pixels in x
    y_pixels = 2648  # number of pixels in y
    PIXEL_SIZE = 0.000833333  # size of the pixel...        
    x_min = 147.39291452904  
    y_max = -29.7000920265888  # x_min & y_max are like the "top left" corner.

    driver = gdal.GetDriverByName('GTiff')

    dataset = driver.Create(
        dst_filename,
        x_pixels,
        y_pixels,
        1,
        gdal.GDT_Float32, )

    dataset.SetGeoTransform((
        x_min,    # 0
        PIXEL_SIZE,  # 1
        0,                      # 2
        y_max,    # 3
        0,                      # 4
        -PIXEL_SIZE))  

    outRasterSRS=osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    dataset.SetProjection(outRasterSRS.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(array)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1)  #If you need to return, remenber to return  also the dataset because the band don`t live without dataset.

new_array_to_raster(Hy_K, 'C:/Users/ifue3702/Documents/Data Au/NSW/Ks_PTF/Ks_Minasny_NN.tif')#runs the function